In [1]:
from dotenv import load_dotenv
load_dotenv(verbose=True)

True

In [2]:
from pathlib import Path
env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)

True

In [3]:
import os
mysql_user = os.getenv('MYSQL_USER')
mysql_pass = os.getenv('MYSQL_PASS')
mysql_host = os.getenv('MYSQL_HOST')
mysql_port = os.getenv('MYSQL_PORT')
mysql_db = os.getenv('MYSQL_DB')

In [4]:
from bs4 import BeautifulSoup as bs
from pprint import pprint
import requests as req
from sqlalchemy import create_engine
from itertools import cycle

In [5]:
engine = create_engine('mysql+mysqldb://{0}:{1}@{2}:{3}'.format(mysql_user,mysql_pass,mysql_host,mysql_port))

In [6]:
dbs = engine.execute('SHOW DATABASES;')
dbs = [x[0] for x in dbs]

In [7]:
if mysql_db not in dbs:
	engine.execute('CREATE DATABASE {0}'.format(mysql_db))
	print('Created database {0}'.format(mysql_db))

In [8]:
engine = create_engine('mysql+mysqldb://{0}:{1}@{2}:{3}/{4}'.format(mysql_user,mysql_pass,mysql_host,mysql_port,mysql_db))
connection = engine.connect()

In [9]:
from sqlalchemy import Column,MetaData,Table,VARCHAR
metadata = MetaData()

In [10]:
stg_persons = Table('stg_persons',metadata,
	Column('name',VARCHAR(1000)),
	Column('lastname',VARCHAR(1000)),
	Column('dni',VARCHAR(1000)))
metadata.create_all(engine)
#print(engine.table_names())

In [11]:
stg_persons = Table('stg_persons',metadata,autoload=True,autoload_with=engine)
#stmt = stg_persons.delete().where(1==1); connection.execute(stmt)

# Scraping

In [12]:
def get_proxies():
	res = req.get('https://free-proxy-list.net')
	soup = bs(res.text,'html.parser')
	trs = soup.find(id='proxylisttable').findAll('tr')
	proxies = []
	for tr in trs:
		tds = tr.findAll('td')#; tds and print(tds)
		tds and proxies.append([td.text for i,td in enumerate(tds) if(i in(0,1,3,6))])

	proxies = list(filter(lambda proxy: proxy[2] in('Brazil','Colombia','Peru') and proxy[3] == 'yes', proxies))
	proxies = ["{0}:{1}".format(proxy[0],proxy[1]) for proxy in proxies]
	return proxies

In [15]:
import random,sys

i = 40000000
while i < 50000000:
	print(i)
	#proxies = cycle(get_proxies()); proxy = next(proxies); print(proxy)
	proxies = get_proxies(); index = random.randint(0,len(proxies)-1); proxy = proxies[index]; print(proxy)
	try:
		res = req.post('https://padron.americatv.com.pe',data={'dni':str(i).zfill(8)},proxies={'https':proxy})
		print(res.ok)
		soup = bs(res.text,'html.parser')
		if(soup.find(id='nameData')):
			name = soup.find(id='nameData').get('value')
			lastname = soup.find(id='lastnameData').get('value')
			dni = soup.find(id='dniData').get('value')
			# pylint: disable=E1120
			stmt = stg_persons.insert().values(name=name,lastname=lastname,dni=dni)
			connection.execute(stmt)
		i += 1
	except:
		print(sys.exc_info())

40000000
190.156.255.86:60623
True
40000001
190.156.255.86:60623
True
40000002
177.185.20.9:52385
True
40000003
191.36.172.66:35491
True
40000004
190.156.255.86:60623
True
40000005
168.227.106.100:33089
True
40000006
190.156.255.86:60623
True
40000007
187.111.192.146:49232
True
40000008
168.121.46.236:39808
(<class 'requests.exceptions.ProxyError'>, ProxyError(MaxRetryError("HTTPSConnectionPool(host='padron.americatv.com.pe', port=443): Max retries exceeded with url: / (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fbbe869ceb8>: Failed to establish a new connection: [Errno 111] Connection refused',)))",),), <traceback object at 0x7fbbe869ea08>)
40000008
161.132.175.136:32356
True
40000009
170.238.133.138:32459
True
40000010
181.129.49.26:53750
(<class 'requests.exceptions.ProxyError'>, ProxyError(MaxRetryError("HTTPSConnectionPool(host='padron.americatv.com.pe', port=443): Max retries exceeded with url: / (

KeyboardInterrupt: 